In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import datetime
import pytz
import pandas as pd

from CardiacMotion.utils.run_helpers import fhm_mesh
from CardiacMotion.utils.run_helpers import Run, get_runs

ModuleNotFoundError: No module named 'CardiacMotion'

In [ ]:
class Repos:
    
    import os    
    
    GWAS = f"{os.environ['HOME']}/01_repos/GWAS_pipeline/"
    CARDIAC = f"{os.environ['HOME']}/01_repos/CardiacCOMA/"
    CARDIAC_MOTION = f"{os.environ['HOME']}/01_repos/CardiacMotionRL"
    CARDIAC_GWAS = f"{os.environ['HOME']}/01_repos/CardiacGWAS/"
    CARDIAC_MOTION_GWAS = f"{os.environ['HOME']}/01_repos/CardiacMotionGWAS/"
    UKBB_HELPERS = f"{os.environ['HOME']}/01_repos/UKBB_helpers"
    

In [ ]:
dosages_df = pd.read_csv(f"{Repos.CARDIAC_MOTION_GWAS}/data/dosages.csv").set_index("ID")

In [ ]:
column_means = dosages_df.mean()
dosages_df = dosages_df.fillna(column_means)

___

In [ ]:
pd.read_csv(f"/home/user/01_repos/CardiacMotionGWAS/data/latent_vector_all_runs_after_Feb2024.csv")

In [ ]:
z_df_merged = pd.read_csv(f"/home/user/01_repos/CardiacMotionGWAS/data/latent_vector_all_runs_after_Feb2024__adj_GBR.csv", sep="\t")
z_df_merged = z_df_merged.set_index("ID")
missing_rows = z_df_merged.apply(lambda x: any((x == -999) | (x == -9)), axis=1)
valid_rows = z_df_merged.index[~missing_rows]
valid_rows = set(dosages_df.index).intersection(valid_rows)

In [ ]:
z_df_merged = z_df_merged.loc[valid_rows]

In [ ]:
dosages_df = dosages_df.loc[valid_rows]

In [ ]:
from scipy.stats import pearsonr, spearmanr

In [ ]:
pval_list = []

for col in tqdm(z_df_merged.columns):    
  for snp in dosages_df.columns:
    
    x = z_df_merged.loc[:, col]
    y = dosages_df.loc[:, snp]
    
    try:
        t, p = pearsonr(x,y)
        pval_row = pd.DataFrame([[col, snp, p]], columns=["phenotype", "snp", "pvalue"])
        # if p < 1e-8: print(col, snp, p)
        pval_list.append(pval_row)
    except Exception as e:
        print(e)
        pass
    
pval_df = pd.concat(pval_list)

In [ ]:
best_pvals = pval_df.sort_values("pvalue").head(115)
best_pvals['region'] = best_pvals.apply(lambda x: snp_to_region[x.snp], axis=1)
best_pvals['chamber'] = best_pvals.phenotype.apply(lambda x: run_to_chamber[x.split("_")[1]])


In [ ]:
best_pvals[best_pvals.region == "chr1_124"]

In [ ]:
from CardiacMotion.utils.run_helpers import Run, get_runs
runs_df = get_runs()

In [ ]:
EXPERIMENT_DICT = {
  "3": "RV",
  "4": "LV",
  "5": "BV",
  "6": "LA",
  "7": "RA",
  "8": "AO",
}

In [ ]:
run_to_chamber = runs_df.set_index("run_id")["experiment_id"].apply(lambda x: EXPERIMENT_DICT[x]).to_dict()

In [ ]:
runs_df['partition'] = runs_df.set_index("run_id")["experiment_id"].apply(lambda x: EXPERIMENT_DICT[x])

In [ ]:
best_pvals.loc[best_pvals.region == "chr5_103"]

In [ ]:
from helpers import EnsembleGWASResults
gwas_results = EnsembleGWASResults(root_dir=f"/mnt/data/workshop/workshop-user1/output/GWAS/All_partitions_spatiotemporal", top_n_runs_per_chamber=None)
snp_to_region = gwas_results.region_assocs_df.set_index("SNP")[["region"]].to_dict()['region']

___

In [ ]:
fpcs_df = pd.read_csv("~/01_repos/CardiacMotionGWAS/results/seed_42/functional_PCs_lvv_lvm_rvm-discovery_10n90__extended_to_61301.csv", delimiter="\t")
fpcs_df = fpcs_df.set_index("ID")
missing_rows = fpcs_df.apply(lambda x: any((x == -999) | (x == -9)), axis=1)
valid_rows = fpcs_df.index[~missing_rows]
valid_rows = set(dosages_df.index).intersection(valid_rows)

fpcs_df = fpcs_df.loc[valid_rows]

In [ ]:
dosages_df = dosages_df.loc[valid_rows]

In [ ]:
from scipy.stats import pearsonr, spearmanr

In [ ]:
pval_list = []

for col in tqdm(fpcs_df.columns):    
  for snp in dosages_df.columns:
    
    x = fpcs_df.loc[:, col]
    y = dosages_df.loc[:, snp]
    
    try:
        t, p = pearsonr(x,y)
        pval_row = pd.DataFrame([[col, snp, p]], columns=["phenotype", "snp", "pvalue"])
        # if p < 1e-8: print(col, snp, p)
        pval_list.append(pval_row)
    except e:
        print(e)
        pass
    
pval_df = pd.concat(pval_list)

In [ ]:
pval_df.sort_values("pvalue")

In [ ]:
from helpers import EnsembleGWASResults
GWAS_RESULTS_DIR = "/mnt/data/workshop/workshop-user1/output/GWAS/All_partitions_spatiotemporal/"
results = EnsembleGWASResults(GWAS_RESULTS_DIR)

In [ ]:
snps_df = results.region_assocs_df

In [ ]:
snps_df = results.loci_summary()


In [ ]:
snps_df = snps_df.reset_index()
snps_dynamic_df = snps_df[snps_df.variable_type == "dynamic"]

In [ ]:
snps_dynamic_df.loc[snps_dynamic_df.groupby("region")["min_P"].idxmin()].sort_values("min_P").head(40)

In [ ]:
merged_df = pd.merge(pval_df, snps_df, left_on="snp", right_on="SNP")
best_pheno = merged_df.loc[merged_df.groupby("snp")["pvalue"].idxmin()]
best_pheno.sort_values("pvalue")